In [6]:
import babelnet as bn
from babelnet import BabelSynsetID, Language
from babelnet.data.relation import BabelPointer

2023-09-02 09:43:25,215 [babelnet.conf] INFO: Loaded configuration from ['/home/giovanni/unimore/2_graph_analytics/tesina/babeldist/remote/babelnet_conf.yml']
2023-09-02 09:43:25,219 [babelnet.api] INFO: BabelNet Remote Procedure Call API v1.1.0


In [7]:
bn.version()

5.0

In [4]:
import sys
import time
sys.path.append('../')
from utils.utils import get_current_logfile_number 
from zerorpc import TimeoutExpired, LostRemote

In [13]:
### Root searcher
visited = set()

max_visits, n = 20000, 0
start_synset_id = 'bn:00015267n'
start_synset = bn.get_synset(BabelSynsetID(start_synset_id))
q = [start_synset]
fname = get_current_logfile_number('candidate_roots')

with open(fname, 'w') as logfile:
    logfile.write(f'start_node={start_synset_id}\n')
    logfile.write(f'max_visits={max_visits}\n')

    start_t = time.time()
    while q and n <= max_visits:
        synset = q.pop(0)
        n += 1
        try:
            hypernym_edges = []
            hypernym_edges = synset.outgoing_edges(BabelPointer.ANY_HYPERNYM)
        except TimeoutExpired:
            logfile.write(f'TimeoutExpired,{synset.id}\n')
        except LostRemote:
            logfile.write(f'LostRemote,{synset.id}\n')

        if hypernym_edges == []:
            print(f'Possible root node: {synset.id}')
            logfile.write(f'candidate_root,{str(synset.id)},{synset.main_sense().full_lemma}\n')
        for edge in hypernym_edges:
            if edge.id_target not in visited:
                try:
                    q.append(bn.get_synset(edge.id_target))
                    visited.add(edge.id_target)
                except TimeoutExpired:
                    logfile.write(f'TimeoutExpired,{synset.id}\n')
                except LostRemote:
                    logfile.write(f'LostRemote,{synset.id}\n')
    end_t = time.time()
    min, sec = divmod(end_t - start_t, 60)
    logfile.write(f'total_time,{int(min)}m,{int(sec)}s')

Possible root node: bn:00076248n


2023-09-02 10:04:24,929 [zerorpc.channel] WARNING: zerorpc.ChannelMultiplexer, unable to route event: <b''> OK {'message_id': b'4af7d398c4ea4022987c96d45de91046', 'v': 3, 'response_to': b'25e9bcd726754fe8ab2ccdb7aac215d0'} [...]


Possible root node: bn:00044576n


2023-09-02 10:07:46,281 [zerorpc.channel] WARNING: zerorpc.ChannelMultiplexer, unable to route event: <b''> OK {'message_id': b'4af7d4a3c4ea4022987c96d45de91046', 'v': 3, 'response_to': b'25e9bde326754fe8ab2ccdb7aac215d0'} [...]


Possible root node: bn:14481752n
Possible root node: bn:00248578n
Possible root node: bn:21705963n
Possible root node: bn:04751152n
Possible root node: bn:16932101n
Possible root node: bn:00031027n
Possible root node: bn:04889559n
Possible root node: bn:14685380n
Possible root node: bn:03245758n
Possible root node: bn:01027542n
Possible root node: bn:02636022n
Possible root node: bn:00894769n
Possible root node: bn:15921369n
Possible root node: bn:04591277n
Possible root node: bn:06171637n
Possible root node: bn:03758703n
Possible root node: bn:01404775n
Possible root node: bn:06162070n
Possible root node: bn:00234465n
Possible root node: bn:15090031n
Possible root node: bn:07669899n
Possible root node: bn:00218497n
Possible root node: bn:02936056n
Possible root node: bn:02638689n
Possible root node: bn:06429342n
Possible root node: bn:07129841n
Possible root node: bn:03843033n
Possible root node: bn:17690782n
Possible root node: bn:05749505n
Possible root node: bn:06760533n
Possible r

In [15]:
for id_lemma in open('../log/candidate_roots.txt', 'r').readlines()[2:-1]:
    id, lemma = id_lemma.strip().split(',')[1:]
    synset = bn.get_synset(BabelSynsetID(id))
    print(f'{id},{lemma},{len(synset.outgoing_edges())},{len(synset.outgoing_edges(BabelPointer.ANY_HYPERNYM))}')

bn:00076248n,taxonomic_group,0,0


2023-09-02 10:27:20,746 [zerorpc.channel] WARNING: zerorpc.ChannelMultiplexer, unable to route event: <b''> OK {'message_id': b'4af7e82ec4ea4022987c96d45de91046', 'v': 3, 'response_to': b'25e9d17026754fe8ab2ccdb7aac215d0'} [...]


bn:00044576n,homo,0,0


2023-09-02 10:29:27,011 [zerorpc.channel] WARNING: zerorpc.ChannelMultiplexer, unable to route event: <b''> OK {'message_id': b'4af7e830c4ea4022987c96d45de91046', 'v': 3, 'response_to': b'25e9d17326754fe8ab2ccdb7aac215d0'} [...]


bn:14481752n,field_of_study,77,0
bn:00248578n,Trade_literature,22,0
bn:21705963n,Religious_calling,46,0
bn:04751152n,production_center,7,0
bn:16932101n,routine,140,0
bn:00031027n,entity,18,0
bn:04889559n,flow_of_matter_and_energy,22,0
bn:14685380n,resource,200,0
bn:03245758n,engineering_process,512,0
bn:01027542n,source_of_information,164,0
bn:02636022n,Thermodynamic_process,427,0
bn:00894769n,task_(computing),40,0
bn:15921369n,adaptation,14,0
bn:04591277n,Work_(human_activity),224,0
bn:06171637n,signaling_system,66,0
bn:03758703n,Mutual_intelligibility,571,0
bn:01404775n,Subatomic_particle,843,0
bn:06162070n,format,24,0
bn:00234465n,Unordered_pair,58,0
bn:15090031n,posture,424,0
bn:07669899n,suprasegmental,75,0
bn:00218497n,Identity_formation,77,0
bn:02936056n,Cellular_component,365,0
bn:02638689n,Course_(medicine),169,0
bn:06429342n,Dokument_normatywny,5,0
bn:07129841n,classification_of_languages,261,0
bn:03843033n,Challenge_(TV_channel),513,0
bn:17690782n,Constant-recursive_sequence